In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta

RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5

In [2]:
 def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

In [3]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund

    def init(self):
        close = self.data.Close.astype(float)  
        volume = self.data.Volume.astype(float)  

        # Calculate RSI and OBV
        self.rsi = self.I(ta.RSI, close, timeperiod=RSI_PERIOD)
        self.obv = self.I(ta.OBV, close, volume)
        self.obv_slope = self.I(pd.Series(self.obv).diff, periods=OBV_PERIOD)

        self.previous_rsi = self.I(pd.Series(self.rsi).shift, 1)
        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage

        # Check for buy signal
        if (self.previous_rsi[-1] < RSI_OVERSOLD and
            self.rsi[-1] >= RSI_OVERSOLD and
            self.obv_slope[-1] > 0):
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            shares_to_buy = (shares_to_buy // 100) * 100
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB', 'VPB', 'FPT', 'REE', 'MSN','VNM'] 

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-14 23:28:45,380 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; u

Results for VCB:
Total investment: 2032.4328653794498
Current Shares: 800
Current Equity: 2779.0339157245635
RoR: 36.734352364732366
--------------------------------------------------


2024-08-14 23:29:01,393 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; u

Results for VPB:
Total investment: 2271.876734681825
Current Shares: 4900
Current Equity: 3654.1007194244607
RoR: 60.84062412550809
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

...

Results for REE:
Total investment: 0.0
Current Shares: 0.0
Current Equity: 0.0
RoR: nan
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for MSN:
Total investment: 2728.9284890271083
Current Shares: 1100
Current Equity: 3115.7245632065783
RoR: 14.17391755536133
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for VNM:
Total investment: 546.205472379969
Current Shares: 200
Current Equity: 560.082219938335
RoR: 2.5405727807707867
--------------------------------------------------


In [3]:
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)
stock_data = Vnstock().stock(symbol='REE').quote.history(start='2019-01-01', end='2024-01-04')
stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
stock_data.set_index('time', inplace=True)
stock_data.index = pd.to_datetime(stock_data.index)

# Merge USD/VND data
stock_data.index = stock_data.index.normalize()
stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
stock_data = stock_data.dropna()
stock_data

2024-08-12 07:33:02,561 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,Open,High,Low,Close,Volume,usd/vnd
time,,,,,,
2019-01-02,17.74,17.74,17.49,0.754074,329830,23.194
2019-01-03,17.49,17.54,17.06,0.738122,588400,23.194
2019-01-04,17.06,17.23,16.80,0.732948,179540,23.194
2019-01-07,17.34,17.43,17.23,0.742865,158080,23.194
2019-01-08,17.23,17.37,17.12,0.738122,191810,23.194
...,...,...,...,...,...,...
2023-12-27,47.84,48.35,47.84,1.984378,527778,24.325
2023-12-28,48.52,48.61,48.18,1.982748,346945,24.345
2023-12-29,48.78,49.21,48.27,2.005777,712798,24.235


In [6]:
stock_data['RSI'] = ta.momentum.RSIIndicator(stock_data['Close'], RSI_PERIOD).rsi()
stock_data['OBV'] = ta.volume.OnBalanceVolumeIndicator(stock_data['Close'], stock_data['Volume']).on_balance_volume()
stock_data['OBV_Slope'] = stock_data['OBV'].diff(periods=OBV_PERIOD)
stock_data['Previous_RSI'] = stock_data['RSI'].shift(1)
stock_data['Previous_RSI'].fillna(0, inplace=True)
def strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df

stock_data = strategy(stock_data)
stock_data

/var/folders/6x/4gbfx9d54wx9s97mvyxfnsy80000gn/T/ipykernel_999/2724411108.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stock_data['Previous_RSI'].fillna(0, inplace=True)


,Open,High,Low,Close,Volume,usd/vnd,RSI,OBV,OBV_Slope,Previous_RSI,Signal
time,,,,,,,,,,,
2019-01-02,17.74,17.74,17.49,0.754074,329830,23.194,NaN,329830,NaN,0.000000,0
2019-01-03,17.49,17.54,17.06,0.738122,588400,23.194,NaN,-258570,NaN,0.000000,0
2019-01-04,17.06,17.23,16.80,0.732948,179540,23.194,NaN,-438110,NaN,0.000000,0
2019-01-07,17.34,17.43,17.23,0.742865,158080,23.194,NaN,-280030,NaN,0.000000,0
2019-01-08,17.23,17.37,17.12,0.738122,191810,23.194,NaN,-471840,NaN,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,47.84,48.35,47.84,1.984378,527778,24.325,43.859944,90166575,535676.0,40.624509,0
2023-12-28,48.52,48.61,48.18,1.982748,346945,24.345,43.555374,89819630,-345254.0,43.859944,0
2023-12-29,48.78,49.21,48.27,2.005777,712798,24.235,48.948478,90532428,750962.0,43.555374,0


In [12]:
stock_data[stock_data["Signal"] == 1]

,Open,High,Low,Close,Volume,usd/vnd,RSI,OBV,OBV_Slope,Previous_RSI,Signal
time,,,,,,,,,,,
2019-01-21,18.31,18.71,18.26,0.804518,280480,23.194,72.653964,890650,632560.0,0.0,1


In [13]:
stock_data[stock_data["Signal"] == -1]

,Open,High,Low,Close,Volume,usd/vnd,RSI,OBV,OBV_Slope,Previous_RSI,Signal
time,,,,,,,,,,,
2019-02-27,20.67,20.67,20.43,0.880490,317890,23.203,69.508545,866890,-425400.0,71.236276,-1
2020-09-29,25.44,25.50,25.19,1.086338,607330,23.188,67.241125,21954450,-645180.0,72.541094,-1
2020-10-02,26.35,26.48,25.22,1.091513,797440,23.188,59.395573,22202690,-623750.0,77.442839,-1
2021-09-07,41.68,41.74,40.35,1.772847,1306800,22.760,64.692520,43991310,-824900.0,75.266858,-1
2022-04-07,53.47,54.22,51.07,2.233447,1025700,22.866,59.346138,68851210,-551800.0,72.905979,-1
